In [2]:
!pip install catboost

In [4]:
!pip uninstall numpy
!pip install numpy
!pip install --force-reinstall catboost

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.11/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.2.4 which i

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from mlxtend.evaluate import PredefinedHoldoutSplit
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

In [98]:
X_full = pd.read_csv('New_Train_Features_Full.csv')
X_test_full = pd.read_csv('New_Test_Features_Full.csv')
y = pd.read_csv('Train_Target.csv').iloc[:,1]

X_full_no = pd.read_csv('New_Train_Features.csv')
X_test_full_no = pd.read_csv('New_Test_Features.csv')

X_no_manual = pd.read_csv('New_Train_Features_No_Selected.csv')
X_test_no_manual = pd.read_csv('New_Test_Features_No_Selected.csv')

X_RFE = pd.read_csv('New_Train_Features_RFE.csv')
X_RFE_test = pd.read_csv('New_Test_Features_RFE.csv')

X_LASSO = pd.read_csv('New_Train_Features_LASSO.csv')
X_LASSO_test = pd.read_csv('New_Test_Features_LASSO.csv')

X_manual = pd.read_csv('New_Train_Features_Selected.csv')
X_manual_test = pd.read_csv('New_Test_Features_Selected.csv')

# Full Dataset:

In [13]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [14]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [30]:
params = {
    "iterations": [500, 1000, 1500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "depth": [4, 6, 8],
    "l2_leaf_reg": [1, 3, 5],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.7, 0.8, 1.0],
}

cat_features = ["Board_SSC", "Board_HSC", "Stream_HSC", "Course_Degree", "Specialization_MBA", "Entrance_Test"]
cat = CatBoostClassifier(random_state = 2025, verbose = 0, cat_features = cat_features)

In [32]:
np.random.seed(2025)
cat_grid = RandomizedSearchCV(cat, param_distributions = params, n_iter = 350, cv = split, n_jobs = -1, scoring = 'f1_macro')
cat_grid.fit(X_temp, y_temp)
print('Best Parameters:', cat_grid.best_params_)

Best Parameters: {'subsample': 1.0, 'learning_rate': 0.01, 'l2_leaf_reg': 1, 'iterations': 1500, 'depth': 4, 'colsample_bylevel': 0.7}


In [37]:
np.random.seed(2025)
cat_model = CatBoostClassifier(subsample = 1.0, learning_rate = 0.01, l2_leaf_reg = 1, iterations = 1500, depth = 4, colsample_bylevel = 0.7, cat_features = cat_features, random_state = 2025, verbose = 0)
cat_model.fit(X_temp, y_temp,)
y_pred = cat_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [38]:
np.random.seed(2025)
cat_model.fit(X_full, y)
y_pred_cat = cat_model.predict(X_test_full)

In [39]:
from google.colab import files

pred_cat = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_cat})

with open('CAT_Full_Cov.csv', 'w') as file:
    pred_cat.to_csv(file, index = False, header = True)

files.download('CAT_Full_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# RFE Dataset:

In [43]:
np.random.seed(2025)
X_train_rfe, X_temp_rfe, y_train_rfe, y_temp_rfe = train_test_split(X_RFE, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_rfe, X_test_rfe, y_val_rfe, y_test_rfe = train_test_split(X_temp_rfe, y_temp_rfe, test_size = 0.5, random_state = 2025, stratify = y_temp_rfe)

In [44]:
np.random.seed(2025)
train_rfe_ind, val_rfe_ind = train_test_split(np.arange(X_temp_rfe.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_rfe)
split_rfe = PredefinedHoldoutSplit(valid_indices = val_rfe_ind)

In [46]:
params = {
    "iterations": [500, 1000, 1500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "depth": [4, 6, 8],
    "l2_leaf_reg": [1, 3, 5],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.7, 0.8, 1.0],
}

cat_features = ["Board_SSC", "Board_HSC", "Stream_HSC", "Course_Degree", "Specialization_MBA"]
cat = CatBoostClassifier(random_state = 2025, verbose = 0, cat_features = cat_features)

In [47]:
np.random.seed(2025)
cat_grid_rfe = RandomizedSearchCV(cat, param_distributions = params, n_iter = 350, cv = split_rfe, n_jobs = -1, scoring = 'f1_macro')
cat_grid_rfe.fit(X_temp_rfe, y_temp_rfe)
print('Best Parameters:', cat_grid_rfe.best_params_)

Best Parameters: {'subsample': 1.0, 'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 4, 'colsample_bylevel': 1.0}


In [51]:
np.random.seed(2025)
cat_model_rfe = CatBoostClassifier(subsample = 1.0, learning_rate = 0.05, l2_leaf_reg = 3, iterations = 1000, depth = 4, colsample_bylevel = 1.0, cat_features = cat_features, random_state = 2025, verbose = 0)
cat_model_rfe.fit(X_temp_rfe, y_temp_rfe)
y_pred_rfe = cat_model_rfe.predict(X_test_rfe)

print(classification_report(y_test, y_pred_rfe))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [63]:
np.random.seed(2025)
cat_model_rfe.fit(X_RFE, y)
y_pred_rfe_cat = cat_model_rfe.predict(X_RFE_test)

In [64]:
from google.colab import files

pred_cat = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_rfe_cat})

with open('CAT_RFE_Cov.csv', 'w') as file:
    pred_cat.to_csv(file, index = False, header = True)

files.download('CAT_RFE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LASSO:

In [65]:
np.random.seed(2025)
X_train_lasso, X_temp_lasso, y_train_lasso, y_temp_lasso = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_lasso, X_test_lasso, y_val_lasso, y_test_lasso = train_test_split(X_temp_lasso, y_temp_lasso, test_size = 0.5, random_state = 2025, stratify = y_temp_lasso)

In [66]:
np.random.seed(2025)
train_lasso_ind, val_lasso_ind = train_test_split(np.arange(X_temp_lasso.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_lasso)
split_lasso = PredefinedHoldoutSplit(valid_indices = val_lasso_ind)

In [67]:
params = {
    "iterations": [500, 1000, 1500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "depth": [4, 6, 8],
    "l2_leaf_reg": [1, 3, 5],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.7, 0.8, 1.0],
}

cat_features = ["Board_SSC", "Board_HSC", "Stream_HSC", "Course_Degree", "Specialization_MBA", "Entrance_Test"]
cat = CatBoostClassifier(random_state = 2025, verbose = 0, cat_features = cat_features)

In [68]:
np.random.seed(2025)
cat_grid_lasso = RandomizedSearchCV(cat, param_distributions = params, n_iter = 350, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
cat_grid_lasso.fit(X_temp_lasso, y_temp_lasso)
print('Best Parameters:', cat_grid_lasso.best_params_)

Best Parameters: {'subsample': 0.8, 'learning_rate': 0.01, 'l2_leaf_reg': 3, 'iterations': 500, 'depth': 6, 'colsample_bylevel': 1.0}


In [70]:
np.random.seed(2025)
cat_model_lasso = CatBoostClassifier(subsample = 0.8, learning_rate = 0.01, l2_leaf_reg = 3, iterations = 500, depth = 6, colsample_bylevel = 1.0, cat_features = cat_features, random_state = 2025, verbose = 0)
cat_model_lasso.fit(X_temp_lasso, y_temp_lasso)
y_pred_lasso = cat_model_lasso.predict(X_test_lasso)

print(classification_report(y_test, y_pred_lasso,zero_division = 1))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89        36
           1       1.00      0.00      0.00         9

    accuracy                           0.80        45
   macro avg       0.90      0.50      0.44        45
weighted avg       0.84      0.80      0.71        45



In [74]:
np.random.seed(2025)
cat_model_lasso.fit(X_LASSO, y)
y_pred_lasso_cat = cat_model_lasso.predict(X_LASSO_test)

In [75]:
from google.colab import files

pred_cat = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_lasso_cat})

with open('CAT_LASSO_Cov.csv', 'w') as file:
    pred_cat.to_csv(file, index = False, header = True)

files.download('CAT_LASSO_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization




In [76]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full_no, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [77]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [78]:
params = {
    "iterations": [500, 1000, 1500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "depth": [4, 6, 8],
    "l2_leaf_reg": [1, 3, 5],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.7, 0.8, 1.0],
}

cat_features = ["Board_SSC", "Board_HSC", "Stream_HSC", "Course_Degree", "Specialization_MBA", "Entrance_Test"]
cat = CatBoostClassifier(random_state = 2025, verbose = 0, cat_features = cat_features)

In [79]:
np.random.seed(2025)
cat_grid = RandomizedSearchCV(cat, param_distributions = params, n_iter = 350, cv = split, n_jobs = -1, scoring = 'f1_macro')
cat_grid.fit(X_temp, y_temp)
print('Best Parameters:', cat_grid.best_params_)

Best Parameters: {'subsample': 1.0, 'learning_rate': 0.01, 'l2_leaf_reg': 1, 'iterations': 1500, 'depth': 4, 'colsample_bylevel': 0.7}


In [80]:
np.random.seed(2025)
cat_model = CatBoostClassifier(subsample = 1.0, learning_rate = 0.01, l2_leaf_reg = 1, iterations = 1500, depth = 4, colsample_bylevel = 0.7, cat_features = cat_features, random_state = 2025, verbose = 0)
cat_model.fit(X_temp, y_temp)
y_pred_cat_no = cat_model.predict(X_test)

print(classification_report(y_test, y_pred_cat_no))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [81]:
np.random.seed(2025)
cat_model.fit(X_full_no, y)
y_pred_cat_no = cat_model.predict(X_test_full_no)

In [82]:
from google.colab import files

pred_cat = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_cat_no})

with open('CAT_Full_No_Cov.csv', 'w') as file:
    pred_cat.to_csv(file, index = False, header = True)

files.download('CAT_Full_No_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization - Manual



In [83]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_no_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [84]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [85]:
params = {
    "iterations": [500, 1000, 1500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "depth": [4, 6, 8],
    "l2_leaf_reg": [1, 3, 5],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.7, 0.8, 1.0],
}

cat_features = ["Board_SSC", "Board_HSC", "Stream_HSC", "Course_Degree", "Specialization_MBA"]
cat = CatBoostClassifier(random_state = 2025, verbose = 0, cat_features = cat_features)

In [86]:
np.random.seed(2025)
cat_grid = RandomizedSearchCV(cat, param_distributions = params, n_iter = 350, cv = split, n_jobs = -1, scoring = 'f1_macro')
cat_grid.fit(X_temp, y_temp)
print('Best Parameters:', cat_grid.best_params_)

Best Parameters: {'subsample': 1.0, 'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 1500, 'depth': 4, 'colsample_bylevel': 0.8}


In [87]:
np.random.seed(2025)
cat_model = CatBoostClassifier(subsample = 1.0, learning_rate = 0.1, l2_leaf_reg = 3, iterations = 1500, depth = 4, colsample_bylevel = 0.8, cat_features = cat_features, random_state = 2025, verbose = 0)
cat_model.fit(X_temp, y_temp)
y_pred_manual = cat_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [90]:
np.random.seed(2025)
cat_model.fit(X_no_manual, y)
y_pred_cat_no_manual = cat_model.predict(X_test_no_manual)

In [91]:
from google.colab import files

pred_cat = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_cat_no_manual})

with open('CAT_No_Manual_Cov.csv', 'w') as file:
    pred_cat.to_csv(file, index = False, header = True)

files.download('CAT_No_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Manual Selection:

In [92]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [93]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [94]:
params = {
    "iterations": [500, 1000, 1500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "depth": [4, 6, 8],
    "l2_leaf_reg": [1, 3, 5],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bylevel": [0.7, 0.8, 1.0],
}

cat_features = ["Board_SSC", "Board_HSC", "Stream_HSC", "Course_Degree", "Specialization_MBA"]
cat = CatBoostClassifier(random_state = 2025, verbose = 0, cat_features = cat_features)

In [95]:
np.random.seed(2025)
cat_grid = RandomizedSearchCV(cat, param_distributions = params, n_iter = 350, cv = split, n_jobs = -1, scoring = 'f1_macro')
cat_grid.fit(X_temp, y_temp)
print('Best Parameters:', cat_grid.best_params_)

Best Parameters: {'subsample': 0.9, 'learning_rate': 0.2, 'l2_leaf_reg': 3, 'iterations': 1500, 'depth': 4, 'colsample_bylevel': 0.8}


In [96]:
np.random.seed(2025)
cat_model = CatBoostClassifier(subsample = 0.9, learning_rate = 0.2, l2_leaf_reg = 3, iterations = 1500, depth = 4, colsample_bylevel = 0.8, cat_features = cat_features, random_state = 2025, verbose = 0)
cat_model.fit(X_temp, y_temp)
y_pred_manual = cat_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99        36
           1       1.00      0.89      0.94         9

    accuracy                           0.98        45
   macro avg       0.99      0.94      0.96        45
weighted avg       0.98      0.98      0.98        45



In [99]:
np.random.seed(2025)
cat_model.fit(X_manual, y)
y_pred_cat_manual = cat_model.predict(X_manual_test)

In [100]:
from google.colab import files

pred_cat = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_cat_manual})

with open('CAT_Manual_Cov.csv', 'w') as file:
    pred_cat.to_csv(file, index = False, header = True)

files.download('CAT_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>